In [ ]:
# This is included for documentation purpose, not a part of this tutorial
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

# Parallel Frames

Let $\mathbf{e}$ and $\mathbf{e}'$ represent two inertial observers. $\mathbf{e}'$ moving  parallel to $\mathbf{e}$ at a constant velocity $v=[v_1\:\:\:v_2\:\:\:v_3]^T$.

We'll animate those frames.

Import following libraries on your notebook

In [ ]:
import plotly.graph_objects as go
import numpy as np

In [ ]:
def create_line_trace(start, end, color='blue', width=2, name='', dash='solid', showlegend=False):
    line_trace = go.Scatter3d(
        x=[start[0], end[0]],
        y=[start[1], end[1]],
        z=[start[2], end[2]],
        mode='lines',
        line=dict(color=color, width=width, dash=dash),
        name=name,
        showlegend=showlegend
    )
    return line_trace

def create_point_trace(point, color='red', size=5, name=''):
    x, y, z = point
    trace = go.Scatter3d(
        x=[x], y=[y], z=[z],
        mode='markers+text',
        marker=dict(color=color, size=size),
        text=[name],
        textposition="top center",
        showlegend=False
    )
    return trace

def create_arrow_trace(start, end, color='blue', name='', showlegend=False):
    # Create arrow shaft trace
    shaft_trace = go.Scatter3d(
        x=[start[0], end[0]], y=[start[1], end[1]], z=[start[2], end[2]],
        mode='lines',
        line=dict(color=color, width=5),
        name=name,
        showlegend=showlegend
    )

    # Create arrowhead trace
    vec = np.array(end) - np.array(start)
    length = np.linalg.norm(vec)
    vec_normalized = vec / length if length > 0 else vec
    arrowhead_size = 0.1 * length

    head_trace = go.Cone(
        x=[end[0]], y=[end[1]], z=[end[2]],
        u=[vec_normalized[0]], v=[vec_normalized[1]], w=[vec_normalized[2]],
        sizemode="absolute", sizeref=arrowhead_size, showscale=False,
        anchor="tip", colorscale=[[0, color], [1, color]]
    )

    return [shaft_trace, head_trace]

def create_3d_layout(title='3D Plot', gridcolor='lightblue', xaxis_title='X Axis', yaxis_title='Y Axis', zaxis_title='Z Axis'):
    layout = go.Layout(
        title=title,
        scene=dict(
            xaxis=dict(showbackground=False, showgrid=True, zeroline=True, zerolinewidth=2, zerolinecolor='rgba(0,0,0,0.5)', showticklabels=True, gridcolor=gridcolor, title=xaxis_title),
            yaxis=dict(showbackground=False, showgrid=True, zeroline=True, zerolinewidth=2, zerolinecolor='rgba(0,0,0,0.5)', showticklabels=True, gridcolor=gridcolor, title=yaxis_title),
            zaxis=dict(showbackground=False, showgrid=True, zeroline=True, zerolinewidth=2, zerolinecolor='rgba(0,0,0,0.5)', showticklabels=True, gridcolor=gridcolor, title=zaxis_title),
        ),
        scene_aspectmode='cube'
    )
    return layout

def create_orthonormal_frame_traces(frame_name='e', origin=[0, 0, 0], length=1, color='blue', rotation_matrix=np.identity(3)):
    # Define standard basis vectors
    i = np.array([1, 0, 0])
    j = np.array([0, 1, 0])
    k = np.array([0, 0, 1])

    # Apply rotation
    i_rot = np.dot(rotation_matrix, i)
    j_rot = np.dot(rotation_matrix, j)
    k_rot = np.dot(rotation_matrix, k)

    # Create traces for rotated axes with arrows
    frame_traces = []
    frame_traces.extend(create_arrow_trace(start=origin, end=np.array(origin) + length * i_rot, color=color, name=f'{frame_name}_x'))
    frame_traces.extend(create_arrow_trace(start=origin, end=np.array(origin) + length * j_rot, color=color, name=f'{frame_name}_y'))
    frame_traces.extend(create_arrow_trace(start=origin, end=np.array(origin) + length * k_rot, color=color, name=f'{frame_name}_z'))

    return frame_traces

def animate_particle(curve_points, particle_name='Particle', particle_color='red', particle_size=6, animation_speed=10):
    # Creating frames for animation
    frames = []
    for i in range(0, len(curve_points), animation_speed):
        frame = go.Frame(data=[go.Scatter3d(
            x=[curve_points[i][0]],
            y=[curve_points[i][1]],
            z=[curve_points[i][2]],
            mode='markers+text',
            marker=dict(color=particle_color, size=particle_size),
            text=[particle_name],
            textposition='top center',
            textfont=dict(size=15)
        )])
        frames.append(frame)

    return frames

def create_particle_animation(curve_points, title='Particle Animation', name='P', origin=[0, 0, 0]):

    # Validate curve_points
    if not isinstance(curve_points, np.ndarray):
        raise ValueError("curve_points must be a NumPy array.")
    if curve_points.size == 0 or curve_points.ndim != 2 or curve_points.shape[1] != 3:
        raise ValueError("curve_points must be a non-empty 2D NumPy array with 3 columns (x, y, z coordinates).")

    traces = []
    frames = []

    x_vals, y_vals, z_vals = zip(*curve_points)

    # Add initial point and origin 'O'
    traces.append(create_point_trace(curve_points[0], color='green', size=8, name=name))
    traces.append(create_point_trace(origin, color='black', size=3, name='O'))

    # Set layout for the figure
    layout = create_3d_layout(title=title, xaxis_title='X Axis', yaxis_title='Y Axis', zaxis_title='Z Axis')

    # Add path trace
    traces.append(go.Scatter3d(
        x=x_vals, y=y_vals, z=z_vals,
        mode="lines",
        line=dict(color="blue", width=2),
        name='Path'
    ))

    # Animate the particle
    frames = animate_particle(curve_points, name, particle_color='green', particle_size=8, animation_speed=1)

    fig = go.Figure(data=traces, layout=layout, frames=frames)

    # Adjust the camera settings
    fig.update_layout(
        scene=dict(
            camera=dict(
                up=dict(x=0, y=0, z=1),  # Sets the up direction (in this case, the z-axis is up)
                center=dict(x=0, y=0, z=0),  # Centers the view on the given coordinates
                eye=dict(x=1, y=-1.25, z=1.25)  # Sets the position of the camera
            ),
            aspectmode='cube'  # Keeps the aspect ratio of the axes fixed
        )
    )

    # Add play and pause buttons
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                buttons=[
                    dict(label="Play",
                          method="animate",
                          args=[None, dict(frame=dict(duration=50, redraw=True), fromcurrent=True)]),
                    dict(label="Pause",
                          method="animate",
                          args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")])
                ]
            )
        ]
    )

    return fig

<div class="alert alert-info">

Note

The functions `create_line_trace`, `create_point_trace`, `create_arrow_trace`, and and others were written in previous tutorials. Please include them in your notebook on top before starting to follow this tutorial. You can download it by clicking the Download icon on the Navigation Bar.

</div>

Since we have explained the syntax and functions going to use here I am leaving the code only. Follow the previous tutorials for better understanding. Hit the play button and see!

In [ ]:
traces = []
frames = []
origin = [0, 0, 0]
origin_prime = [5, 5, 5]
interval = np.linspace(0, 50, 100)

# Add origin 'O'
traces.append(create_point_trace(origin_prime, color='black', size=3, name="O'"))

# Add origin 'O'
traces.append(create_point_trace(origin, color='black', size=3, name='O'))

# Create orthonormal frame traces
frame_traces = create_orthonormal_frame_traces(frame_name="e'", origin=origin_prime, length=10, color='green')
traces.extend(frame_traces)

# Create orthonormal frame traces
frame_traces = create_orthonormal_frame_traces(frame_name='e', origin=origin, length=10, color='red')
traces.extend(frame_traces)


traces.append(create_line_trace(start=origin, end=[50, 0, 0], color='blue', width=3, dash='dash', name='e', showlegend=True))
traces.append(create_line_trace(start=[0, 5, 5], end=[50, 5, 5], color='purple', width=3, dash='dash', name="e'", showlegend=True))

velocity_prime = 0.5  # Velocity for frame 'e''
velocity = 1  # Velocity for frame 'e'

# Animation
for i in range(0, len(interval), 5):
    new_origin_prime = [origin_prime[0] + interval[i] * velocity_prime, origin_prime[1], origin_prime[2]]
    new_origin = [origin[0] + interval[i] * velocity, origin[1], origin[2]]

    frame_data = [
        # Move origin prime 'O''
        create_point_trace(new_origin_prime, color='black', size=3, name="O'"),
        # Move origin 'O'
        create_point_trace(new_origin, color='black', size=3, name='O'),
    ]

    # Move the orthonormal frames
    frame_traces_prime = create_orthonormal_frame_traces(frame_name="e'", origin=new_origin_prime, length=10, color='green')
    frame_traces = create_orthonormal_frame_traces(frame_name='e', origin=new_origin, length=10, color='red')

    frame_data.extend(frame_traces_prime)
    frame_data.extend(frame_traces)

    frames.append(go.Frame(data=frame_data))

# Set layout for the figure
layout = create_3d_layout(title='Parallel Frames', xaxis_title='e1 Axis', yaxis_title='e2 Axis', zaxis_title='e3 Axis')

fig = go.Figure()
fig = go.Figure(data=traces, layout=layout, frames=frames)

axis_range = [0, 60]
# Adjust the camera settings
fig.update_layout(
    scene=dict(
            xaxis=dict(range=axis_range),
            yaxis=dict(range=axis_range),
            zaxis=dict(range=axis_range),
        camera=dict(
            up=dict(x=0, y=0, z=1),  # Sets the up direction (in this case, the z-axis is up)
            center=dict(x=0, y=0, z=0),  # Centers the view on the given coordinates
            eye=dict(x=1, y=-1.25, z=1.25)  # Sets the position of the camera
        ),
        aspectmode='cube'  # Keeps the aspect ratio of the axes fixed
    )
)

# Add play and pause buttons
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(label="Play",
                      method="animate",
                      args=[None, dict(frame=dict(duration=50, redraw=True), fromcurrent=True)]),
                dict(label="Pause",
                      method="animate",
                      args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")])
            ]
        )
    ]
)

fig.show()